In [1]:
from pathlib import Path

In [2]:
import ethicml
from ethicml.algorithms.inprocess import GPyT, GPyTDemPar, GPyTEqOdds, LR, SVM, Agarwal, Kamiran, Kamishima, LRCV, ZafarEqOpp
from ethicml.evaluators import evaluate_models, CrossValidator, run_metrics
from ethicml.data import Compas, Adult, load_data
from ethicml.metrics import Accuracy, ProbPos, TPR, TNR, AbsCV
from ethicml.preprocessing import train_test_split
from ethicml.visualisation.plot import plot_mean_std_box

In [3]:
# YOU DON'T HAVE TO UNDERSTAND THE CODE IN THIS CELL
# we only tell the model where the "run.py" is and where the python executable is
code_dir = Path('..')
def gp(**kwargs):
    return GPyT(code_dir=code_dir, **kwargs)
def gp_dp(**kwargs):
    return GPyTDemPar(code_dir=code_dir, **kwargs)
def gp_eo(**kwargs):
    return GPyTEqOdds(code_dir=code_dir, **kwargs)

In [4]:
# tnr_race_False = 0.724
# tnr_race_True = 0.702
# tnr_sex_True = 0.724
# tnr_sex_False = 0.744
# tnr_in_true_race = 0.71
# tnr_in_false_race = 0.74
# tnr_in_true_sex = 0.72
# tnr_in_false_sex = 0.77

tnr_in_false_race = 0.74
tnr_in_false_sex = 0.74

In [23]:
# specify flags for GP
gp_flags = dict(epochs=70, length_scale=1.2, use_loo=False, iso=False, cov='GridInterpolationKernel')

In [24]:
algos = []

# algos += [gp(s_as_input=True, flags=gp_flags)]
algos += [gp(s_as_input=False, flags=gp_flags)]
# algos += [gp_dp(epochs=70, s_as_input=True)]

for tpr in [0.6,0.7,0.8,0.9]:
#     algos += [gp_eo(s_as_input=True, tnr1=tnr_in_true_race, tnr0=tnr_in_true_race, tpr0=tpr, tpr1=tpr, flags=gp_flags)]
    algos += [gp_eo(s_as_input=False, tnr1=tnr_in_false_race, tnr0=tnr_in_false_race, tpr0=tpr, tpr1=tpr, flags=gp_flags)]

baselines = [
#      LR(),
     SVM(),
#     Agarwal(fairness="EqOd"),
#     Kamiran(),
#     Kamishima(),
#    ZafarEqOpp(),
]
algos += baselines

In [25]:
data = [
#      Compas("Race"),
      Compas("Sex"),
#     Adult("Race"),
#     Adult("Sex"),
]

In [38]:
results = evaluate_models(
    datasets=data,
    inprocess_models=algos,
    metrics=[Accuracy(), TPR(), TNR()],
    per_sens_metrics=[TPR(), TNR()], 
    repeats=5,
    proportional_splits=True,                      # <- new!
    delete_prev=True,  # delete previous results
)















  0%|          | 0/30 [00:00<?, ?it/s]













  0%|          | 0/30 [00:00<?, ?it/s, model=GPyT_in_False, dataset=Compas Sex, transform=no_transform, repeat=0]

Failure: ['../run.py', '--inf', 'Variational', '--data', 'sensitive_from_numpy', '--dataset_path', '/tmp/tmpe30xc2lm/data.npz', '--cov', 'GridInterpolationKernel', '--mean', 'ZeroMean', '--optimizer', 'Adam', '--lr', '0.05', '--model_name', 'local', '--batch_size', '4933', '--epochs', '70', '--eval_epochs', '100000', '--summary_steps', '100000', '--chkpt_epochs', '100000', '--save_dir', '/tmp/tmpe30xc2lm', '--plot', '', '--logging_steps', '1', '--gpus', '0', '--preds_path', 'predictions.npz', '--num_samples', '20', '--optimize_inducing', 'True', '--length_scale', '1.2', '--sf', '1.0', '--iso', 'False', '--num_samples_pred', '2000', '--s_as_input', 'False', '--num_inducing', '1245', '--manual_seed', '888', '--lik', 'BaselineLikelihood', '--use_loo', 'False'] (pid = 25070)
/mnt/archive/fairness/fair-gpytorch/fairgp/datasets.py:85: RuntimeWarning: invalid value encountered in true_divide
  return np.where(max_per_feature > 1e-7, unnormalized / max_per_feature, unnormalized)
/mnt/archive/f

RuntimeError: The script failed. Supplied arguments: ['../run.py', '--inf', 'Variational', '--data', 'sensitive_from_numpy', '--dataset_path', '/tmp/tmpe30xc2lm/data.npz', '--cov', 'GridInterpolationKernel', '--mean', 'ZeroMean', '--optimizer', 'Adam', '--lr', '0.05', '--model_name', 'local', '--batch_size', '4933', '--epochs', '70', '--eval_epochs', '100000', '--summary_steps', '100000', '--chkpt_epochs', '100000', '--save_dir', '/tmp/tmpe30xc2lm', '--plot', '', '--logging_steps', '1', '--gpus', '0', '--preds_path', 'predictions.npz', '--num_samples', '20', '--optimize_inducing', 'True', '--length_scale', '1.2', '--sf', '1.0', '--iso', 'False', '--num_samples_pred', '2000', '--s_as_input', 'False', '--num_inducing', '1245', '--manual_seed', '888', '--lik', 'BaselineLikelihood', '--use_loo', 'False'] with exec: /mnt/archive/fairness/py_env/bin/python3

In [22]:
# import pandas as pd
# results = pd.read_csv("results/Compas Sex_no_transform.csv").set_index(["dataset", "transform", "model", "repeat"])

plot_mean_std_box(results, Accuracy(), TPR())
# figs_plots = plot_mean_std_box(results, "Accuracy", "TPR_race_0/race_1")
# figs_plots[0][0]

[(<Figure size 1800x1200 with 1 Axes>,
  <matplotlib.axes._subplots.AxesSubplot at 0x7ff1a4ac2710>),
 (<Figure size 1800x1200 with 1 Axes>,
  <matplotlib.axes._subplots.AxesSubplot at 0x7ff1a48b2128>)]

In [13]:
train, test = train_test_split(load_data(Adult()))

In [ ]:
# Fair grid search
primary = Accuracy()
fair_measure = AbsCV()
hyperparams = dict(C=[1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7])
lr_cv = CrossValidator(LR, hyperparams, folds=5)
lr_cv.run(train, measures=[primary, fair_measure])
lr_cv.best_hyper_params(primary)

In [ ]:
lr_best_acc = lr_cv.results.get_best_result(primary)
lr_best_fair = lr_cv.results.get_best_in_top_k(primary, fair_measure, top_k=3)
print("best accuracy:", lr_best_acc)
print("best fair(+accuracy):", lr_best_fair)

In [ ]:
lr_best_acc_model = LRCV()
results = evaluate_models(
    datasets=[Adult()],
    inprocess_models=[LR(**lr_best_fair.params), lr_best_acc_model],
    metrics=[Accuracy(), ProbPos(), TPR(), TNR()],
    per_sens_metrics=[ProbPos(), TPR(), TNR()], 
    repeats=3,
    delete_prev=True,  # delete previous results
)

In [ ]:
plot_mean_std_box(results, Accuracy(), ProbPos())